Andrea Palladio - Villa Badoer
================

Working again on Villa Badoer as started in workshop_05.
Aim of this final workshop is to completing the model.

Villa Badoer is a villa in Fratta Polesine in the Veneto region of northern Italy. It was designed in 1556 by Andrea Palladio for the Venetian noble Francesco Badoer, and built between 1557 and 1563, on the site of a medieval castle which guarded a bridge across a navigable canal. This was the first time Palladio used his fully developed temple pediment in the facade of a villa.
Villa Badoer has been part since 1996 of the UNESCO World Heritage Site "City of Vicenza and the Palladian Villas of the Veneto".

Constructed and inhabited in 1556, the villa therefore functioned for the management of the fields and was simultaneously a visible sign of the “feudal” presence, so to speak, of Badoer in the territory: it is not coincidental that the building rises on the site of an ancient medieval castle.
Palladio succeeded in uniting within one effective synthesis these dual meanings, joining the majestic manor house to the two barchesse (farm wings) bent into semicircles, which screen the stables and other agricultural annexes.

![VillaBadoer](./images/villa.jpg)

In [177]:
from larlib import *
from math import floor
import csv
import os

# Reading data
As mentioned in workshop_05 I will assign 3d-points instead of getting data from the filesystem. This time I will put all data in a separate file, for readability.
This will prevent problems regarding paths etc. this approach may come in handy for fine tune points ;)

In [178]:
def larModel(path2lines, openPath=0):
    """
    larModel("mypath")
    :param path2file: path to the .lines file
    :return: tuple. A graph model representing Vertices and Edges.
             Edges are lists of indices to Vertices.
    """
    with open(path2lines, "rb") as file:
        lines = csv.reader(file, delimiter=",")

        mapping = {}
        v = []
        ev = []
        index = 0
        
        # indexing of the vertices and approximation
        for entry in lines:
            x1 = math.floor(float(entry[0])*10)/10
            y1 = math.floor(float(entry[1])*10)/10
            x2 = math.floor(float(entry[2])*10)/10
            y2 = math.floor(float(entry[3])*10)/10

            if not(mapping.has_key((x1,y1))):
                index += 1
                mapping[(x1,y1)] = index
                v.append([x1,y1])

            if not(mapping.has_key((x2,y2))):
                index += 1
                mapping[(x2,y2)] = index
                v.append([x2,y2])

            ev.append([mapping[(x1,y1)], mapping[(x2,y2)]])
        
        #this will close the path by linking the last node with the first one
        if (ev[-1][1] != ev[0][0] and not(openPath)):
            lastEdge = [ev[-1][1], ev[0][0]]
            ev.append(lastEdge)
        
        #print (lastEdge[0] == ev[-2][1]) #True
        #print (lastEdge[1] == ev[0][0]) #True
        #print ev[-1] == lastEdge
        
        
    return v,ev

# Work pipeline
- Divide et impera: I will start off with splitting the structure.
  - For every part of Villa Badoer a .csv and a .lines will be produced (https://inkscape.org)
  - Alternatively, if pre-existing pyplasm model for polyhedra are sufficient, then those will be used
- Given a .lines file for a certain part of the structure, produce a graph (V,EV) as lar model
- Given a part of the villa, draw its plant
- Given a plant use cartesian product to build walls etc.
- Figure out a way to use symmetries

In [179]:
from data import pyData # Pythonized data from .lines

# Base
Probably as a result of exploiting the substructures of the medieval castle, the manor house of the villa rises on a high basement, and recalls illustrious precedents like the Villa Medici at Poggio a Caiano by Giuliano da Sangallo, and the not far distant Villa dei Vescovi at Luvigliano by Giovanni Maria Falconetto.


The base on which the main structure lies is composed of two layers, base1 and base2, plus a frontal base adjacent to the stairs.
Base2 is placed upon base1, frontbase situates in front of base1 and base2 

In [180]:
#reload(pyData)

### Base 1

In [181]:
VBase1,EVBase1 = larModel(os.getcwd() + "/data/base1.lines")

In [182]:
VBase1 = pyData.VBase1
EVBase = pyData.EVBase1


In [183]:
VBase1

[[51.4, 83.4],
 [51.4, 120.7],
 [54.9, 120.7],
 [54.9, 115.7],
 [54.9, 115.7],
 [56.7, 115.7],
 [56.7, 120.7],
 [102.6, 120.7],
 [102.6, 115.6],
 [104.3, 115.6],
 [104.3, 120.7],
 [107.7, 120.7],
 [107.7, 83.4]]

### Base 2

In [184]:
VBase2,EVBase2 = larModel(os.getcwd() + "/data/base2.lines")

In [185]:
VBase2 = pyData.VBase2
EVBase2 = pyData.EVBase2

# Frontal base

In [186]:
VBaseF,EVBaseF = larModel(os.getcwd() + "/data/frontbase2.lines")

In [187]:
VBaseF = pyData.VBaseF
EVBaseF = pyData.EVBaseF

# Drawing the wire-frame plant

Again from workshop_05

In [188]:
def drawPlant(verts, edges, pr = 0):
    """
    drawPlant(V,EV) returns the wire-flame plant of the input, the lar model.
    It is clearly a 2d model, this implies that it does not handle space for doors, windows, stairs etc.
    
    :param verts: list of vertices in form [X, Y]
    :param edges: list of edges in form [indexofVertA, indexofVertB]
    :return: array of POLILINEs
    """
    plant = []
    for e in edges:
        if (pr):
            print ([verts[e[0]-1], verts[e[1]-1]]),"\n"
        plant.append(POLYLINE([verts[e[0]-1], verts[e[1]-1]]))
    return plant

In [189]:
base1plant = drawPlant(VBase1, EVBase1, pr=0)

In [190]:
base2plant = drawPlant(VBase2, EVBase2)


In [191]:
baseFplant = drawPlant(VBaseF, EVBaseF)
#VIEW(STRUCT(baseFplant))

# Building the bases

Scheme: First, I will build every part of the hole base (base1, base2 etc.) then I will use UNION to make a unique base out of each of its part.

Key functions:
- SOLIDIFY(polygon): it joins inner points of both concave and convex polygons
- UNION(polyhs): as the name suggets, it merges input polyhedra


I will assume steps have a height of 20cm and as we can see from the following image, the base has 9 + 5 steps

![steps](./images/gradini.jpg)

### Base 1

In [192]:
base1 = SOLIDIFY(STRUCT(base1plant))
base1 = PROD([base1, Q(0.20*9)])

In [193]:
#VIEW(base1)

### Base 2

In [194]:
base2 = SOLIDIFY(STRUCT(base2plant))
base2 = T(3)(0.20*9)(PROD([base2, Q(0.20*5)]))

In [195]:
#VIEW(base2)

### Frontal Base

In [196]:
baseF = SOLIDIFY(STRUCT(baseFplant))
baseF = PROD([baseF, Q(0.20*14)])

In [197]:
#VIEW(baseF)

## Final base with back stairs
Stairs can also be seen on the back of base1 and 2

In [198]:
def stairs(N,stepsize):
    """
    stairs build a basic, full stairway (no space between steps and the ground)
    :param N: number of steps
    :param stepsize: array of length 3. stepsize contains cartesian coordinate used to model a single step
    """
    sx,sy,sz = stepsize
    V,FV = larCuboids([1,1])
    step = S([1,2])([sx,sy])(STRUCT(MKPOLS((V,FV))))
    step =  steps = T(3)(sz*(N-1))(PROD([step,INTERVALS(sz)(1)]))
    for i in range(1,N):
        nextStep = T(3)(-sz*i)(S(2)(1+i)(step))
        steps = STRUCT([steps,nextStep])
    return steps

In [199]:
"""
Left-hand side back stairs

size depens on the svg/lines data.
It is redundant, in a way, to calculate it as a difference, but this way it gives the author more control.

I decided to model steps so that they starts from the origin [0,0,0], an approach I did not used for other structures.
This way, code will be more reusable. I will need to model various stairs.
"""
nStepsBack = 14 #recurrent value, as Villa Badoer has the same steps number for every stairway

bssX = 56.65 - 54.9
bssY = (117.7 - 115.7)/5.5
bssZ = 0.20 #height of a single step
backStepSize = [bssX, bssY, bssZ]

In [200]:
myStairs = stairs(nStepsBack, backStepSize)

## The whole base

- Base1
- Base2
- Frontal base
- Stairs on the back

In [201]:
bricksBase = './textures/bricks.jpeg'
orangeBricks = './textures/orangeBricks.jpg'

In [202]:
base1 = TEXTURE(bricksBase)(base1)
base2 = TEXTURE(bricksBase)(base2)
baseF = TEXTURE(orangeBricks)(baseF)

leftBackStairs = TEXTURE(bricksBase)(T([1,2])([54.93,115.7])(myStairs))
rightBackStairs = TEXTURE(bricksBase)(T([1,2])([102.55,115.60])(myStairs))

base = STRUCT([base1,base2, baseF, leftBackStairs, rightBackStairs])

In [203]:
VIEW(base)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x18150f5690> >

![base](./images/base.png)

![base_2](./images/base_2.png)

![base_3](./images/base_3.png)

# The main structure
Note: Some sources cite a basement beneth the main floor, yet no images or descriptions seem to be available. Because of that, I decided not to model a multiple floor structure until I find proper material on which to base a valid model.

## Floor
The floor lies on a base which is thinner then the ones we have seen before: it is slightly thinner than base2 (which is 0.20*5), let's assume it is 0.2*3 m thick

In [204]:
VbaseVilla,EVbaseVilla = larModel(os.getcwd() + "/data/mainStrBase.lines", openPath=0)

In [205]:
VbaseVilla = pyData.VbaseVilla

In [206]:
EVbaseVilla = pyData.EVbaseVilla

In [207]:
#EVbaseVilla

In [208]:
villaBasePlant = drawPlant(VbaseVilla, EVbaseVilla)

""" total height of the base of the villa is approximately 10 steps, thus 0.20*10 """

villaBase1 = MULTEXTRUDE(OFFSET([1.1,1.1])(SOLIDIFY(STRUCT(villaBasePlant))))(0.20)
villaBase2 = MULTEXTRUDE(OFFSET([0.7,0.7])(STRUCT(villaBasePlant)))(0.20*8)
villaBase3 = MULTEXTRUDE(OFFSET([1.1,1.1])(SOLIDIFY(STRUCT(villaBasePlant))))(0.20)

villaBase = STRUCT([TOP([TOP([villaBase1, villaBase2]), villaBase3])])

In [209]:
VIEW(villaBase)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181515e060> >

In [210]:
villaFloor = MULTEXTRUDE(OFFSET([0.7,0.7])(SOLIDIFY(STRUCT(villaBasePlant))))(0.06)

## Inner walls.
I would like to use something different from the boolean operation DIFFERENCE to create space for doors. I will stack two model one on top of the other, the former one contains spaces, the latter one does not.

In [211]:
VinnWalls,EVinnWalls = larModel(os.getcwd() + "/data/innerwalls.lines", openPath=1)

In [212]:
VinnWallsTOP,EVinnWallsTOP = larModel(os.getcwd() + "/data/innerwallsTOP.lines", openPath=1)

In [213]:
VinnWallsTOP = pyData.VinnWallsTOP

In [214]:
EVinnWallsTOP = pyData.EVinnWallsTOP

In [215]:
innWallsPlantHoled = STRUCT(drawPlant(VinnWalls, EVinnWalls))
innWallsPlantHoled = MULTEXTRUDE(OFFSET([0.7,0.7])(STRUCT([innWallsPlantHoled])))(2.5)

In [216]:
innWallsPlantTOP = STRUCT(drawPlant(VinnWallsTOP, EVinnWallsTOP))
innWallsPlantTOP = MULTEXTRUDE(OFFSET([0.7,0.7])(STRUCT([innWallsPlantTOP])))(3.5)

In [217]:
innWalls = STRUCT([TOP([innWallsPlantHoled, innWallsPlantTOP])])
VIEW(innWalls)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x18150f5e40> >

In [218]:
travertineHEX = "#E2DECD"

In [219]:
innWalls = HEX(travertineHEX)(innWalls)
VIEW(innWalls)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x18150f5cc0> >

![walls1](./images/walls.png)

![walls2](./images/walls2.png)

![walls3](./images/walls3.png)

In [220]:
parquet = './textures/parquet.jpg'

# Inner walls, floor and base
Let's now give a look inside the villa at current state 

In [221]:
villaBase = HEX(travertineHEX)(villaBase)
innWalls = HEX(travertineHEX)((innWalls))
villaFloor = TEXTURE(parquet)(villaFloor)
habitation = STRUCT([TOP([TOP([villaBase, villaFloor]), innWalls])])
VIEW(habitation)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x18150f5db0> >

![inside](./images/inside.png)

![inside2](./images/inside2.png)

In [222]:
#VIEW(TOP([base, habitation]))

# Windows
Few kinds of windows:
- lower ones are placed on villaBase.
- big green windows are more "central" on walls height
- small green windows are place at the very top of the side walls

![side](./images/sideVillaFoto.jpg)

In [223]:
woodHEX = "#8B5A2B"
steelGreyHEX = "#43464B"
glassTexture = './textures/glass.jpeg'
greenWindowTexture = './textures/greenWindow.jpeg'

### lower windows

In [224]:
windowStructure = SKEL_1(STRUCT([PROD([INTERVALS(0.20*3)(2), INTERVALS(0.20)(1)])]))
windowStructure = HEX(woodHEX)(STRUCT([OFFSET([0.03,0.03, 0.03])(PROD([Q(0.01), windowStructure]))]))

glassSize = SIZE([1,2,3])(windowStructure) # [0.03999999910593033, 0.6299999952316284, 0.23000000417232513]
glassSize[0] /= 4
glass = TEXTURE(glassTexture)(CUBOID(glassSize))

In [225]:
windowGrid = SKEL_1(STRUCT([PROD([INTERVALS(0.62)(6), INTERVALS(0.22)(4)])]))
windowGrid = HEX(steelGreyHEX)(STRUCT([OFFSET([0.01,0.01, 0.01])(PROD([Q(0.01), windowGrid]))]))

window = STRUCT([RIGHT([window2glass,windowGrid])])
VIEW(window)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x182ff8d600> >

### big green windows

In [226]:
bigGrWindowSize = [0.03999999910593033, 0.6299999952316284, 3*0.23000000417232513]
bigGrWindow = TEXTURE(greenWindowTexture)(CUBOID(bigGrWindowSize))

VIEW(bigGrWindow)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x182ff8da50> >

### small green windows

In [227]:
smallGrWindowSize = SIZE([1,2,3])(windowStructure)
smallGrWindow = TEXTURE(greenWindowTexture)(CUBOID(smallGrWindowSize))

VIEW(STRUCT([smallGrWindow]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x182ff96480> >

In [228]:
def placeWindows():
    #TODO
    return